<a href="https://colab.research.google.com/github/mssomie/Deep-Neural-Network/blob/main/Concrete_Strength_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Artificial Neural Network from Scratch


In [13]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Define activation functions

Logistics Function

In [14]:
def logisitics_function (x):
  return 1/(1+np.exp(-x))


RELU Function


In [15]:
def RELU(x):
  return np.maximum(0,x)


Hyperbolic Function

In [16]:
def hyberbolic_tangent(x):
    function = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    derivative = 1 - function ** 2
    return function, derivative

In [17]:
print(logisitics_function(10))
print(RELU(10))
print(hyberbolic_tangent(10))

0.9999546021312976
10
(0.9999999958776926, 8.244614768671e-09)


# Define Fitness Function


In [18]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

In [19]:
def fitness_function(y_true, y_pred):
    return 1/MSE(y_true, y_pred)

# Get Data

In [23]:
# Get dataset
url = "https://archive.ics.uci.edu/static/public/165/concrete+compressive+strength.zip"
response = requests.get(url)
with open("concrete_strength.zip", "wb") as file:
    file.write(response.content)

NameError: name 'requests' is not defined

In [21]:
# Unzip dataset
!unzip concrete+compressive+strength.zip  -d /content

unzip:  cannot find or open concrete+compressive+strength.zip, concrete+compressive+strength.zip.zip or concrete+compressive+strength.zip.ZIP.


In [22]:
# Load data into dataframe
concrete_data = pd.read_excel(r"/content/Concrete_Data.xls")

# View first five entries
concrete_data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/Concrete_Data.xls'

In [9]:
# Check data information
print(concrete_data.shape)
concrete_data.info()

(1030, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1030 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1030 non-null   float64
 7   Age (day)                                              1030 non-null   int64  
 8   Concrete compressive strength(MPa, mega

In [10]:
# view summary statistics
concrete_data.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


# Data Preprocessing

In [11]:
# Check for missing data
concrete_data.isnull().any().sum()

0

In [12]:
# Check for duplicates
print(concrete_data.duplicated().any())
print("Number of duplicate rows:", concrete_data.duplicated().sum())
duplicate_rows = concrete_data[concrete_data.duplicated()]
duplicate_rows

True
Number of duplicate rows: 25


,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
77,425.0,106.3,0.0,153.5,16.5,852.1,887.1,3,33.398217
80,425.0,106.3,0.0,153.5,16.5,852.1,887.1,3,33.398217
86,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3,35.301171
88,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3,35.301171
91,362.6,189.0,0.0,164.9,11.6,944.7,755.8,3,35.301171
100,425.0,106.3,0.0,153.5,16.5,852.1,887.1,7,49.201007
103,425.0,106.3,0.0,153.5,16.5,852.1,887.1,7,49.201007
109,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7,55.895819
111,362.6,189.0,0.0,164.9,11.6,944.7,755.8,7,55.895819
123,425.0,106.3,0.0,153.5,16.5,852.1,887.1,28,60.294676


In [13]:
# Handle Duplicates
concrete_data.drop_duplicates(inplace = True)
print(concrete_data.duplicated().any())

False


**Split Data Into Training and Testing Sets**


In [12]:
x = concrete_data.drop(columns=['Concrete compressive strength'])
y = concrete_data['Concrete compressive strength']
concrete_data_train, concrete_data_test = train_test_split(x,y, test_size=0.3, random_state=7)

NameError: name 'concrete_data' is not defined

# Network


In [14]:
# TODO: fix for multiple hidden layers
class Multilayer_perceptron:

   # Initialize ANN
   def __init__(self, hidden_layers_neurons, activation_function, input_layer_neurons=8,  output_layer_neuron=1, bias_input =1):
       self.input_layer_neurons = input_layer_neurons
       self.hidden_layers_neurons = hidden_layers_neurons
       self.output_layer_neuron = output_layer_neuron
       self.weight = []
       self.bias = []

       self.activation_function = activation_function

       # Input layer to hidden layer
       self.weight.append(np.random.rand(hidden_layers_neurons[0],input_layer_neurons))
       self.bias.append(np.random.rand(hidden_layers_neurons[0],bias_input))

      # Hidden layer to hidden layer
       for i in range(1, len(hidden_layers_neurons)):
        self.weight.append(np.random.rand(hidden_layers_neurons[i], hidden_layers_neurons[i-1]))
        self.bias.append(np.random.rand(hidden_layers_neurons[i], bias_input))


       # Hidden layer to output layer
       self.weight.append(np.random.rand(output_layer_neuron,hidden_layers_neurons[-1]))
       self.bias.append(np.random.rand(output_layer_neuron,bias_input))

   def forward_pass(self, input):
        # Make input a 2D array
        input = np.array(input).reshape(-1,1)

       # using Z = Wx + b
        z_hidden=[]
        a_hidden=[]
        for i in range(len(self.weight)):
            if i == 0:
                z = np.dot(self.weight[i], input) + self.bias[i]
            else:
                z = np.dot(self.weight[i], a_hidden[i-1]) + self.bias[i]
            z_hidden.append(z)
            a_hidden.append(self.activation_function(z))

        return a_hidden[-1]

In [15]:
# Test the Multilayer_perceptron class
def test_multilayer_perceptron():
    # Define test parameters
    input_neurons = 8
    hidden_layers = [10, 8, 6]
    output_neurons = 1
    activation_func = RELU  # Using the RELU function defined earlier

    # Create an instance of Multilayer_perceptron
    mlp = Multilayer_perceptron(hidden_layers, activation_func, input_neurons, output_neurons)

    # Check if the network structure is correct
    assert len(mlp.weight) == len(hidden_layers) + 1, "Incorrect number of weight matrices"
    assert len(mlp.bias) == len(hidden_layers) + 1, "Incorrect number of bias vectors"

    # Check dimensions of weight matrices and bias vectors
    assert mlp.weight[0].shape == (hidden_layers[0], input_neurons), "Incorrect shape of first weight matrix"
    for i in range(1, len(hidden_layers)):
        assert mlp.weight[i].shape == (hidden_layers[i], hidden_layers[i-1]), f"Incorrect shape of weight matrix {i}"
    assert mlp.weight[-1].shape == (output_neurons, hidden_layers[-1]), "Incorrect shape of output weight matrix"

    for i, neurons in enumerate(hidden_layers):
        assert mlp.bias[i].shape == (neurons, 1), f"Incorrect shape of bias vector {i}"
    assert mlp.bias[-1].shape == (output_neurons, 1), "Incorrect shape of output bias vector"

    # Test forward pass
    input_data = np.random.rand(input_neurons, 1)  # Create random input data
    output = mlp.forward_pass(input_data)

    assert output.shape == (output_neurons, 1), "Incorrect output shape"

    print("All tests passed!")

# Run the test
test_multilayer_perceptron()


All tests passed!


**Particle Swarm Optimization**

In [3]:
class Particle_Swarm_Optimization:
    def __init__(self, swarm_size, dimensions, fitness_function, alpha, beta, gamma, delta, epsilon):
        self.swarm_size = swarm_size
        self.dimensions = dimensions
        self.fitness_function = fitness_function
        self.particles_fittest = []
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon



    def initiailize_swarm(self):
        # Initialize the particles with random positions
        self.particles = np.random.rand(self.swarm_size, self.dimensions)
        self.velocities = np.zeros((self.swarm_size, self.dimensions))

        # Initialize the personal best positions as the particles' initial positions
        self.personal_best_positions = self.particles.copy()
        self.global_best_position = self.particles[0].copy()
        
        for i in range (self.swarm_size):
            self.point_a = np.random.rand(self.swarm_size, self.dimensions)
            self.point_b = np.random.rand(self.swarm_size, self.dimensions)
            self.vector = self.point_b - self.point_a 
            self.velocities =  self.vector * 1/2
        pass

    
    def assess_fitness(self, particle):
        pass

    
    def update_velocity(self, particle):
        pass

    def update_position(self, particle):
        pass

    def optimize(self, max_iterations):

        pass
        
